# Proyecto Open Data I
## Radares, y su eficiencia en la CAM
### Recopilación, limpieza y tratamiento de los datos
Este cuaderno pretende enseñar el proceso de limpieza de los datos relativos a los radares en la CAM
_Paula Gómez Lucas, Alejandro Majado Martínez_

In [ ]:
# Importar librerías
import os
import pandas as pd

A continuación, se muestra la clase que está compuesta de todos los métodos que se encargan de la limpieza y transformación de los datos

In [ ]:
class CSVDataLoader:
    """
    A class for loading and cleaning CSV data from a specified folder path.

    Attributes:
    -----------
    folder_path : str
        The path to the folder containing the CSV files to be loaded.

    data : dict
        A dictionary containing the loaded CSV data, where the keys are the file names and the values are the corresponding dataframes.
    """

    def __init__(self, folder_path):
        """
        Initializes a CSVDataLoader object with the specified folder path.

        Parameters:
        -----------
        folder_path : str
            The path to the folder containing the CSV files to be loaded.
        """
        self.folder_path = folder_path
        self.data = {}
        self.filename = []
        self.keys = []

    def load_data(self):
        """
        Loads CSV data from the specified folder path into a dictionary.

        Returns:
        --------
        None
        """
        csv_files = [f for f in os.listdir(self.folder_path) if f.endswith('.csv')]
        folders = ("datasets/actuacionesBomberos", "datasets/estaciones", "datasets/accidentalidad")
        for folder in folders:
            df = None
            for file in os.listdir(folder):
                filepath = folder + "/" + file
                df1 = pd.read_csv(filepath, sep=';', encoding='utf-8', low_memory=False)
                df = pd.concat([df, df1])
            self.data[str(folder)] = df

        for file_name in csv_files:
            file_path = os.path.join(self.folder_path, file_name)
            try:
                df = pd.read_csv(file_path, sep=';', encoding='latin-1', low_memory=False)
                self.data[str(file_name)] = df
                self.filename.append(file_name)
            except Exception as e:
                print(f"Error al leer {file_name}: {str(e)}")

        for value in self.data.keys():
            self.keys.append(value)

    def clean_data(self):
        """
        Cleans the loaded CSV data by renaming columns, removing whitespace, dropping null values and duplicates, and converting date columns to datetime format.

        Returns:
        --------
        None
        """
        columna_borrar = "Unnamed"
        for df in self.data:
            for j in self.data[df].columns:
                if columna_borrar in j:
                    while j in self.data[df].columns:
                        self.data[df] = self.data[df].drop(j, axis=1)
                        self.data[df] = self.data[df].dropna(how='all', axis=0)
                        
            self.data[df] = self.data[df].rename(columns = lambda x: x.strip().lower().replace(' ', '_'))
            self.data[df] = self.data[df].map(lambda x: x.strip() if isinstance(x, str) else x)
            self.data[df] = self.data[df].dropna(how='all', axis=0)
            self.data[df] = self.data[df].drop_duplicates()
            self.data[df] = self.data[df].loc[:, ~self.data[df].columns.duplicated()]
            self.data[df].columns = map(str.upper, self.data[df].columns)

            if 'FECHA' in self.data[df].columns:
                self.data[df]['FECHA'] = pd.to_datetime(self.data[df]['FECHA'], format='%d/%m/%Y')

#           num_cols = self.data[i].select_dtypes(include='number').columns
#           for col in num_cols:
#               self.data[i][col] = self.data[i][col].fillna(self.data[i][col].mean())

    def get_info(self, filename):
        print(self.data[filename].isnull().sum())
        print(self.data[filename].info())
        
    def get_nan_columns(self):
        j = 0
        for i in self.data:
            print(self.keys[j])
            self.get_info(i)
            j+=1
      
    def get_cleaned_data(self):
        """
        Returns the cleaned CSV data as a dictionary.

        Returns:
        --------
        dict
            A dictionary containing the cleaned CSV data, where the keys are the file names and the values are the corresponding dataframes.
        """
        return self.data

Una vez está definida la clase con sus métodos, procedemos a declarar las variables que nos permiten trabajar con ello

In [ ]:
folder_path = "datasets"
data_loader = CSVDataLoader(folder_path)

La siguiente función carga los datos de los csv a los dataframes

In [ ]:
data_loader.load_data()

Limpiamos los datos eliminando las columnas autogeneradas con NaNs, renombramos las columnas para que sean uniformes (minúsculas y con barra bajas), eliminamos las filas de NaNs, eliminamos las filas duplicadas, formateamos todas las variables fecha para que sean consistentes (dd/mm/aaaa), sustituimos los NaNs de las variables numéricas con la media correspondiente a su variable.

In [ ]:
data_loader.clean_data()
data = data_loader.get_cleaned_data()

Por último, para confirmar que los datos se han cargado bien, utilizamos el método get_nan_columns para ver cuántos datos en cada columna quedan nulos, así como usamos el método info() de pandas para ver un resumen de todas las columnas y comprobamos también que todo el formateo de las columnas se ha realizado sin problema.

In [ ]:
data_loader.get_nan_columns()

Efectivamente, se realiza sin problemas. El siguiente paso es el análisis de los datasets columna a columna para revisar qué método usar para rellenar los datos faltantes dependiendo de cada atributo.  

### Sustitución de valores faltantes
Los datasets actuaciones bomberos y estaciones no tienen datos faltantes, por lo que sólo nos queda trabajar con los otros 8 datasets, que podemos observar aquí:

In [ ]:
for i in data.keys():
    print(i)

Empecemos con accidentalidad: 

In [ ]:
data_loader.get_info('datasets/accidentalidad')
data_loader.data['datasets/accidentalidad']

Hay algunos datos faltantes que tiene sentido que lo sean, y podemos sustituir por un buzzword de algún tipo que nos haga saber que se trate de esto, como lo es que en un accidente en el que no se han producido lesiones, la lesividad sea nula y tampoco haya código de la misma, y como ambas cifras coinciden, es lógico pensar que se trata de las mismas situaciones. Podemos sustituir entonces todos los NaNs de Lesividad faltantes por 'Ninguna' y el código por -1.0.

In [ ]:
data_loader.data['datasets/accidentalidad']

Por otro lado, positivo en droga tiene valor sólo si daba positivo, por lo que rellenar los valores faltantes con 0 es lo más lógico (siendo 0 negativo en droga). 

Número, código de distrito, tipo de accidente, coordenadas (x e y), son atributos a los que sólo les falta un dato cada uno, por lo que no es representativo esta falta de datos y podemos rellenarlos con el valor más habitual. 

Por último, donde queda dilema es en positivo alcohol, tipo de vehículo y estado meteorológico. En esta situación, lo más apropiado es ver si, relacionando estos atributos con algún otro, es más probable que los atributos valgan uno u otro valor.

- Estado meteorológico. Hay 7 valores posibles: despejado, lluvia débil, lluvia intensa, granizando, nevando, nublado, se desconoce. Aquí, por lo tanto, hay 3 vías de actuación:
    - Rellenar con "se desconoce", i.e.: ser fieles a lo que se sabe, reducir la proporción de datos artificiales (hay un 11% de datos faltantes), solución sencilla.
    - Rellenar con el valor más frecuente: despejado (representa el 75% de los datos), i.e.: solución con datos artificiales más sencilla.
    - Rellenar con valores aleatorios según la proporción en la que aparecen los datos, i.e.: el 75% de los datos faltantes se rellenan arbitrariamente con "Despejado".

    Lo que mejor preserva los datos es, rellenar con "se desconoce", pues la variable existe previamente.

- Tipo de vehículo. Hay 34 valores posibles: Ambulancia SAMUR, autobús EMT, autobús, autobús articulado, autobús articulado EMT, autocaravana, bicicleta, bicicleta EPAC (pedaleo asistido), camión de bomberos, camión rígido, ciclo, ciclomotor, ciclomotor de dos ruedas L1e-B, cuadriciclo ligero, cuadriciclo no ligero, furgoneta, maquinaria de obras, microbús <= 17 plazas, moto de tres ruedas > 125cc, moto de tres ruedas hasta 125cc, motocicleta > 125cc, motocicleta hasta 125cc, otros vehículos con motor, otros vehículos sin motor, patinete no eléctrico, remolque, semirremolque, sin especificar, todo terreno, tractocamión, tren/metro, turismo (68%), VMU eléctrico, vehículo articulado. En esta variable hay 0.6% de valores faltantes, lo cual no es significativo, i.e.: la sustitución que elijamos tendrá menos repercusión en el estudio final. Aquí, por lo tanto, hay 2 vías de actuación:
    - Rellenar con "sin especificar", i.e.: solución sencilla y descriptiva pero que puede dar lugar a interpretaciones erróneas, pues puede haber sido otro tipo de vehículo que no se había registrado.
    - Rellenar con el valor más probable según otro atributo (por ejemplo, código de lesividad).
    
    La solución más apropiada es rellenar con el valor más probable según código de lesividad, por lo que vamos a ver primero cómo se relacionan ambos atributos y después rellenaremos los valores faltantes con el valor más probable.

In [ ]:
data_loader.get_info('datasets/accidentalidad')

Como podemos comprobar, no quedan datos faltantes. Pasamos a Actuaciones Bomberos:

In [ ]:
data_loader.get_info('datasets/actuacionesBomberos')

Este dataset no tiene datos faltantes, pasamos al siguiente. Estaciones:

In [ ]:
data_loader.get_info('datasets/estaciones')

Este dataset tampoco tiene datos faltantes. Pasamos a la evolución histórica de las direcciones:

In [ ]:
data_loader.get_info('DireccionesEvolucionHistorica_20231004.csv')

Como podemos observar, faltan datos de las columnas VIA_SQC (secuencia de la denominación), con 371584 NaNs; VIA_PAR (partícula de la denominación), con 17997; CALIFICADOR (del número de la policía), con 322826; FECHA_DE_BAJA (de la dirección), con 211657; y TIPO_NDP (tipo de número: portal, garaje, fachada, etc.) con 16248.

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv']

La secuencia de denominación (VIA_SQC) es solo NaNs, por lo que lo más lógico es eliminar la columna directamente

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv'].drop('VIA_SQC', axis=1)

Ahora, pasamos a la partícula de denominación (VIA_PAR), hay 7 valores distintos de entre la lista posible de 20 (DE, EL, A, DEL, LA, AL, DE LA, LAS, A LA, DE LAS, LO, A LAS, DE LO, LOS, A LO, DE LO, DE LOS, POR EL, A LOS, POR LA), de los cuales el primero es A LA y el último es DEL. Por lo tanto, parece que no se ha rellenado con Null o espacio vacío aquellas direcciones en las que no hay partícula:

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['VIA_PAR'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['VIA_PAR'].fillna(' ')

Fecha de baja, si no hay valor, es que está aún de alta, así que también sustituimos por un valor que represente que sigue dado de alta:

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['FECHA_DE_BAJA'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['FECHA_DE_BAJA'].fillna('--/--/----')

Para Tipo de número de policía, rellenaremos con el dato más habitual: PORTAL

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['TIPO_NDP'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['TIPO_NDP'].fillna(data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['TIPO_NDP'].mode())

In [ ]:
data_loader.get_info('DireccionesEvolucionHistorica_20231004.csv')

Efectivamente no quedan Missing Values. Pasamos a las direcciones vigentes:

In [ ]:
data_loader.get_info('DireccionesVigentes_20231004.csv')

Faltan valores en Vía Par (3381) y en Calificador (178838), los sustituimos con el mismo criterio que en el dataset anterior:

In [ ]:
data_loader.data['DireccionesVigentes_20231004.csv']['VIA_PAR'] = data_loader.data['DireccionesVigentes_20231004.csv']['VIA_PAR'].fillna(' ')

In [ ]:
# mismo metodo

In [ ]:
data_loader.get_info('DireccionesVigentes_20231004.csv')

Pasamos al dataset estrella: radares.

In [ ]:
data_loader.get_info('RADARES_FIJOS_vDTT.csv')
data_loader.data['RADARES_FIJOS_vDTT.csv']



Tabla de frecuencia relativa de accidentes por distritos. En esta tabla se obtiene la frecuencia de accidentes en todos los distritos de madrid, esto hace que se pueda obtener también el tráfico de coches en el distrito. Aunque no sea una relación proporcional, a mayor número de accidentes supone un tráfico mayor en la zona.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
accidentes = data_loader.data['datasets/accidentalidad']
frecuencia = accidentes['DISTRITO'].value_counts()
accidentes_df = pd.DataFrame(frecuencia)
accidentes_df.columns = ['Frecuencia absoluta']
accidentes_df['Frecuencia relativa'] = 100*accidentes_df['Frecuencia absoluta'] / len(accidentes)
frecuencia_rel_acumulada = accidentes_df['Frecuencia relativa'].cumsum()
accidentes_df['Frecuencia relativa acumulada'] = frecuencia_rel_acumulada
#por problemas de longitud de bytes, faltan decimales en la suma y no da el 100% exacto
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_title('Distribución de accidentes por distrito')
ax.bar(accidentes_df.index, accidentes_df['Frecuencia absoluta'], color='blue')
ax2 = ax.twinx()
ax2.plot(accidentes_df.index, accidentes_df['Frecuencia relativa acumulada'], color='red', marker='o', ms = 5)
ax2.yaxis.set_major_formatter(PercentFormatter())
ax.tick_params(axis='y', color = 'blue')
ax2.tick_params(axis='y', color = 'red')
ax.set_xticklabels(accidentes_df.index, rotation=90)
plt.show()
